In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import heapq
import time
import threading

# Initial graph data
graph = {
    "Rock Garden": {"Sukhna Lake": 1.5, "Sector 17 Plaza": 2.8},
    "Sukhna Lake": {"Rock Garden": 1.5, "Rose Garden": 3.2},
    "Rose Garden": {"Sukhna Lake": 3.2, "Sector 17 Plaza": 1.8},
    "Sector 17 Plaza": {"Rose Garden": 1.8, "Elante Mall": 4.5, "Rock Garden": 2.8},
    "Elante Mall": {"Sector 17 Plaza": 4.5, "ISBT Sector 43": 5.0},
    "ISBT Sector 43": {"Elante Mall": 5.0, "PGI Chandigarh": 3.5},
    "PGI Chandigarh": {"ISBT Sector 43": 3.5}
}

positions = {
    "Rock Garden": (100, 100),
    "Sukhna Lake": (250, 80),
    "Rose Garden": (350, 150),
    "Sector 17 Plaza": (200, 200),
    "Elante Mall": (400, 250),
    "ISBT Sector 43": (300, 300),
    "PGI Chandigarh": (150, 300),
}

def dijkstra(graph, start, end):
    queue = [(0, start, [])]
    visited = set()

    while queue:
        (cost, node, path) = heapq.heappop(queue)

        if node in visited:
            continue

        visited.add(node)
        path = path + [node]

        if node == end:
            return cost, path

        for neighbor, weight in graph.get(node, {}).items():
            if neighbor not in visited:
                heapq.heappush(queue, (cost + weight, neighbor, path))

    return float("inf"), []

class TripPlannerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Chandigarh Trip Planner")
        self.root.geometry("800x650")

        self.canvas = tk.Canvas(root, width=600, height=400, bg="white")
        self.canvas.pack(pady=10)

        # Control panel
        control_frame = tk.Frame(root)
        control_frame.pack()

        self.locations = list(graph.keys())
        self.start_var = tk.StringVar()
        self.end_var = tk.StringVar()

        ttk.Label(control_frame, text="Start:").grid(row=0, column=0)
        ttk.Combobox(control_frame, textvariable=self.start_var, values=self.locations, width=18).grid(row=0, column=1)

        ttk.Label(control_frame, text="End:").grid(row=0, column=2)
        ttk.Combobox(control_frame, textvariable=self.end_var, values=self.locations, width=18).grid(row=0, column=3)

        tk.Button(control_frame, text="Find Shortest Path", command=self.find_path, bg="#4CAF50", fg="white").grid(row=0, column=4, padx=5)
        tk.Button(control_frame, text="Animate Path", command=self.animate_path, bg="#2196F3", fg="white").grid(row=0, column=5)

        self.result_label = tk.Label(root, text="", font=("Helvetica", 12), wraplength=700)
        self.result_label.pack(pady=5)

        # Add/Remove Controls
        modify_frame = tk.Frame(root)
        modify_frame.pack(pady=10)

        self.new_place_var = tk.StringVar()
        self.connect_to_var = tk.StringVar()
        self.dist_var = tk.StringVar()

        tk.Label(modify_frame, text="New Place:").grid(row=0, column=0)
        tk.Entry(modify_frame, textvariable=self.new_place_var, width=15).grid(row=0, column=1)

        tk.Label(modify_frame, text="Connect To:").grid(row=0, column=2)
        tk.Entry(modify_frame, textvariable=self.connect_to_var, width=15).grid(row=0, column=3)

        tk.Label(modify_frame, text="Distance:").grid(row=0, column=4)
        tk.Entry(modify_frame, textvariable=self.dist_var, width=8).grid(row=0, column=5)

        tk.Button(modify_frame, text="Add Spot", command=self.add_location, bg="#009688", fg="white").grid(row=0, column=6, padx=5)
        tk.Button(modify_frame, text="Remove Spot", command=self.remove_location, bg="#f44336", fg="white").grid(row=0, column=7)

        self.draw_map()

    def draw_map(self, path=[]):
        self.canvas.delete("all")
        drawn_edges = set()

        for node in graph:
            for neighbor in graph[node]:
                edge = tuple(sorted([node, neighbor]))
                if edge in drawn_edges:
                    continue
                drawn_edges.add(edge)

                x1, y1 = positions.get(node, (0, 0))
                x2, y2 = positions.get(neighbor, (0, 0))
                mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2

                color = "red" if (
                    path and node in path and neighbor in path and abs(path.index(node) - path.index(neighbor)) == 1
                ) else "gray"

                self.canvas.create_line(x1, y1, x2, y2, fill=color, width=2)
                self.canvas.create_text(mid_x, mid_y, text=f"{graph[node][neighbor]} km", fill="blue", font=("Arial", 9, "bold"))

        for name, (x, y) in positions.items():
            self.canvas.create_oval(x-10, y-10, x+10, y+10, fill="lightblue")
            self.canvas.create_text(x, y-15, text=name, font=("Arial", 9, "bold"))

    def find_path(self):
        start, end = self.start_var.get(), self.end_var.get()

        if not start or not end:
            messagebox.showwarning("Missing Info", "Select both start and end.")
            return
        if start == end:
            self.result_label.config(text="You're already there!")
            return

        distance, path = dijkstra(graph, start, end)

        if distance == float("inf"):
            self.result_label.config(text="No path found.")
        else:
            self.result_label.config(text=f"Path: {' → '.join(path)}\nTotal Distance: {distance} km")
            self.draw_map(path)

    def animate_path(self):
        start, end = self.start_var.get(), self.end_var.get()
        if not start or not end:
            return

        distance, path = dijkstra(graph, start, end)
        if not path or distance == float("inf"):
            return

        def animate():
            for i in range(len(path)-1):
                self.draw_map(path[:i+2])
                time.sleep(0.8)
            self.result_label.config(text=f"Animated Path: {' → '.join(path)}\nTotal Distance: {distance} km")

        threading.Thread(target=animate).start()

    def add_location(self):
        new_place = self.new_place_var.get().strip()
        connect_to = self.connect_to_var.get().strip()
        try:
            dist = float(self.dist_var.get())
        except ValueError:
            messagebox.showerror("Invalid Input", "Distance must be a number.")
            return

        if not new_place or not connect_to:
            messagebox.showwarning("Missing Info", "Fill all fields to add a spot.")
            return
        if connect_to not in graph:
            messagebox.showerror("Invalid Location", f"{connect_to} doesn't exist.")
            return

        graph.setdefault(new_place, {})[connect_to] = dist
        graph[connect_to][new_place] = dist

        # Auto-place new spot
        x, y = positions[connect_to]
        positions[new_place] = (x + 50, y + 50)

        self.locations = list(graph.keys())
        self.update_dropdowns()
        self.draw_map()

    def remove_location(self):
        to_remove = self.new_place_var.get().strip()
        if to_remove not in graph:
            messagebox.showerror("Invalid Spot", f"{to_remove} doesn't exist.")
            return

        del graph[to_remove]
        for neighbors in graph.values():
            neighbors.pop(to_remove, None)
        positions.pop(to_remove, None)

        self.locations = list(graph.keys())
        self.update_dropdowns()
        self.draw_map()

    def update_dropdowns(self):
        for widget in self.root.winfo_children():
            if isinstance(widget, ttk.Combobox):
                widget["values"] = self.locations

# Run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = TripPlannerApp(root)
    root.mainloop()
